In [ ]:
# ========================================
# ESTRAZIONE CHECKSUM DA TUTTI I MESSAGGI CON CHECKSUM NEL DBC
# ========================================

import pandas as pd
import cantools
from python.psa_checksum import psa_checksum  # Assumendo che questa libreria sia disponibile nell'ambiente

print("🚀 INIZIO ESTRAZIONE CHECKSUM PER TUTTI I MESSAGGI\n")

# Configurazione
LOG_FILE = 'logs/aa0ad8ba95ff270c|00000017--62c22bb216.csv'  # File di log principale
DBC_FILE = 'dbc/psa_aee2010_r3_fixed2.dbc'  # File DBC

# 1. CARICA DBC
print(f"📂 Carico DBC: {DBC_FILE}...")
try:
    dbc = cantools.database.load_file(DBC_FILE, strict=False)
    print(f"✅ DBC caricato\n")
except FileNotFoundError:
    print(f"❌ File {DBC_FILE} non trovato!")
    exit()

# 2. CARICA CSV (una sola volta, per tutti i messaggi)
print(f"\n📂 Carico CSV: {LOG_FILE}...")
df = pd.read_csv(LOG_FILE)
print(f"✅ Caricato: {len(df)} righe\n")

# 3. TROVA TUTTI I MESSAGGI CON CHECKSUM NEL DBC
messages_with_checksum = []
for msg in dbc.messages:
    checksum_signals = [sig for sig in msg.signals if 'CHECKSUM' in sig.name.upper() or 'CRC' in sig.name.upper()]
    if checksum_signals:
        messages_with_checksum.append((msg, checksum_signals))

print(f"✅ Trovati {len(messages_with_checksum)} messaggi con checksum nel DBC\n")

# 4. CICLO SU OGNI MESSAGGIO CON CHECKSUM
for msg, checksum_signals in messages_with_checksum:
    MESSAGE_ID = msg.frame_id
    
    print("=" * 80)
    print(f"PROCESSO MESSAGGIO 0x{MESSAGE_ID:03X} ({msg.name})")
    print("=" * 80)
    
    # Mostra definizione messaggio
    print(f"Nome: {msg.name}")
    print(f"ID: 0x{MESSAGE_ID:03X} ({MESSAGE_ID})")
    print(f"Lunghezza: {msg.length} bytes")
    print(f"Segnali: {len(msg.signals)}")
    print()
    
    print("📋 SEGNALI NEL MESSAGGIO:")
    print("-" * 80)
    for sig in msg.signals:
        print(f"  {sig.name:30s} | Start: {sig.start:3d} | Length: {sig.length:2d} bit | Scale: {sig.scale} | Offset: {sig.offset}")
        if 'CHECKSUM' in sig.name.upper() or 'CRC' in sig.name.upper():
            print(f"     ⚡ QUESTO È UN CHECKSUM!")
    
    print()
    print(f"✅ Trovati {len(checksum_signals)} checksum:")
    for cs in checksum_signals:
        print(f"   - {cs.name}: bit {cs.start}, lunghezza {cs.length} bit")
    print()
    
    # 5. TROVA IL BUS CORRETTO PER QUESTO MESSAGGIO
    possible_buses = [0, 1, 2, 3]  # Assumi bus possibili (espandi se necessario)
    BUS = None
    df_filtered = None
    for b in possible_buses:
        temp_filtered = df[(df['bus'] == b) & (df['addr'] == f'0x{MESSAGE_ID:x}')].copy()
        if len(temp_filtered) > 0:
            BUS = b
            df_filtered = temp_filtered
            break
    
    if BUS is None:
        print(f"❌ Nessun messaggio trovato per 0x{MESSAGE_ID:03X} su nessun bus!")
        continue
    
    print(f"🔍 Messaggi trovati su bus {BUS}: {len(df_filtered)}")
    print()
    
    # 6. DECODIFICA E ESTRAI CHECKSUM
    print("🔄 Decodifica messaggi ed estrazione checksum...\n")
    
    decoded_messages = []
    for idx, row in df_filtered.iterrows():
        try:
            time = row['time']
            data_hex = row['data']
            
            # Prepara bytes
            if data_hex.startswith('0x'):
                data_hex = data_hex[2:]
            if len(data_hex) % 2:
                data_hex = '0' + data_hex
            data_bytes = bytes.fromhex(data_hex)
            
            # Decodifica
            decoded = dbc.decode_message(MESSAGE_ID, data_bytes)
            
            # Aggiungi info base
            decoded['time'] = time
            decoded['raw_hex'] = data_hex
            decoded['raw_bytes'] = ' '.join([data_hex[i:i+2].upper() for i in range(0, len(data_hex), 2)])
            
            # Verifica TUTTI i checksum
            all_checksums_ok = True
            
            for cs_sig in checksum_signals:
                # Calcola checksum - CREA COPIA FRESCA OGNI VOLTA
                data_array = bytearray(data_bytes)  # Copia fresca per ogni checksum!
                calculated = psa_checksum(MESSAGE_ID, cs_sig, data_array)
                
                # Estrai checksum dal messaggio
                extracted = decoded.get(cs_sig.name, None)
                
                # Converti NamedSignalValue in int
                if hasattr(extracted, 'value'):
                    extracted = int(extracted.value)
                else:
                    extracted = int(extracted) if extracted is not None else 0
                    
                # Verifica
                is_ok = (calculated == extracted)
                all_checksums_ok = all_checksums_ok and is_ok
                
                # Salva con nome univoco per ogni checksum
                decoded[f'{cs_sig.name}_extracted'] = extracted
                decoded[f'{cs_sig.name}_calculated'] = calculated
                decoded[f'{cs_sig.name}_ok'] = is_ok
            
            # Flag globale: OK solo se TUTTI sono OK
            decoded['all_checksums_ok'] = all_checksums_ok
            
            decoded_messages.append(decoded)
            
        except Exception as e:
            print(f"⚠️  Errore decodifica a t={time:.3f}s: {e}")
    
    if not decoded_messages:
        print("❌ Nessun messaggio decodificato!")
        continue
    
    df_decoded = pd.DataFrame(decoded_messages)
    print(f"✅ Decodificati {len(df_decoded)} messaggi\n")
    
    # 7. MOSTRA RISULTATI (simile al codice originale, ma abbreviato per brevità)
    print("=" * 80)
    print("MESSAGGI CON CHECKSUM")
    print("=" * 80)
    print()
    
    N = 20  # Mostra primi 20
    print(f"📋 Primi {N} messaggi:\n")
    
    for idx, row in df_decoded.head(N).iterrows():
        time = row['time']
        raw_bytes = row['raw_bytes']
        all_ok = row['all_checksums_ok']
        
        # Status globale
        status = "✓ OK" if all_ok else "✗ FAIL"
        
        print(f"[{time:8.3f}s] {status:6s} | {raw_bytes}")
        
        # Mostra ogni checksum su riga separata
        for cs in checksum_signals:
            extr_val = row[f'{cs.name}_extracted']
            calc_val = row[f'{cs.name}_calculated']
            
            # Converti in int gestendo NamedSignalValue
            try:
                extr = int(float(extr_val))
            except:
                extr = int(extr_val.value) if hasattr(extr_val, 'value') else int(extr_val)
                
            try:
                calc = int(float(calc_val))
            except:
                calc = int(calc_val.value) if hasattr(calc_val, 'value') else int(calc_val)
            
            ok_symbol = "✓" if row[f'{cs.name}_ok'] else "✗"
            print(f"   {cs.name:30s}: Estratto=0x{extr:X}  Calcolato=0x{calc:X}  [{ok_symbol}]")
        
        # Altri segnali (opzionale, abbreviato)
        print()
    
    # 8. STATISTICHE CHECKSUM (simile al codice originale)
    print(f"\n{'=' * 80}")
    print("📊 STATISTICHE CHECKSUM")
    print("=" * 80)
    
    all_ok_count = df_decoded['all_checksums_ok'].sum()
    all_fail_count = len(df_decoded) - all_ok_count
    
    print(f"Totale messaggi:       {len(df_decoded)}")
    print(f"Tutti checksum OK:     {all_ok_count} ({all_ok_count/len(df_decoded)*100:.1f}%)" if len(df_decoded) > 0 else "N/A")
    print(f"Almeno 1 FAIL:         {all_fail_count} ({all_fail_count/len(df_decoded)*100:.1f}%)" if len(df_decoded) > 0 else "N/A")
    print()
    
    # Statistiche per ogni checksum
    for cs in checksum_signals:
        print(f"--- {cs.name} ---")
        ok_count = df_decoded[f'{cs.name}_ok'].sum()
        fail_count = len(df_decoded) - ok_count
        print(f"  OK:   {ok_count} ({ok_count/len(df_decoded)*100:.1f}%)" if len(df_decoded) > 0 else "N/A")
        print(f"  FAIL: {fail_count} ({fail_count/len(df_decoded)*100:.1f}%)" if len(df_decoded) > 0 else "N/A")
        print()
    
    if all_fail_count > 0:
        print(f"⚠️  Esempi di messaggi con checksum errati (primi 5):")
        fails = df_decoded[~df_decoded['all_checksums_ok']].head(5)
        for idx, row in fails.iterrows():
            print(f"\n   t={row['time']:.3f}s: {row['raw_bytes']}")
            for cs in checksum_signals:
                if not row[f'{cs.name}_ok']:
                    extr_val = row[f'{cs.name}_extracted']
                    calc_val = row[f'{cs.name}_calculated']
                    # Prova diversi modi per convertire
                    try:
                        extr = int(float(extr_val))
                        calc = int(float(calc_val))
                    except:
                        extr = extr_val
                        calc = calc_val
                    print(f"      {cs.name}: Estratto={extr}, Calcolato={calc}")
        print()
    
    # Qui potresti aggiungere il salvataggio CSV per ogni messaggio, simile al codice originale

print("\n🎉 COMPLETATO!")